## Import resources and display image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install opencv-python==3.4.2.16

     |████████████████████████████████| 25.0MB 139kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [4]:
pip install opencv-contrib-python==3.4.2.16

     |████████████████████████████████| 30.6MB 111kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [5]:
!git clone https://github.com/vbalnt/tfeat.git

Cloning into 'tfeat'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 224 (delta 0), reused 0 (delta 0), pack-reused 221
Receiving objects: 100% (224/224), 30.80 MiB | 36.33 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [6]:
cd tfeat

/content/tfeat


## Extract Keypoint and decriptors

In [7]:
import cv2
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import torchvision as tv
import phototour
import torch
from tqdm import tqdm 
import numpy as np
import torch.nn as nn
import math 
import tfeat_model
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import tfeat_utils


tfeat = tfeat_model.TNet()
models_path = '/content/tfeat/pretrained-models/'
net_name = 'tfeat-liberty'
tfeat.load_state_dict(torch.load(os.path.join(models_path,net_name+".params")))
tfeat.cuda()
tfeat.eval()

# Datasets
tiso = 'drive/MyDrive/3D Shabnam/Tiso/Images/'
portello = 'drive/MyDrive/3D Shabnam/Portello/Images/'
fountain= 'drive/MyDrive/3D Shabnam/Fountain/images/'
castle = '/content/drive/MyDrive/3D Shabnam/Castle/images/'

# Define Descriptors and keypoints

def BRIEF(image):
  star = cv2.xfeatures2d.StarDetector_create()
  brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
  kp = star.detect(image,None)
  kp, des = brief.compute(image, kp)
  return des,kp


def ORB(image):
  orb = cv2.ORB_create()
  kp, des = orb.detectAndCompute(image,None)
  return des,kp

def TFeat(image):
  training_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
  brisk = cv2.BRISK_create()
  kp, des = brisk.detectAndCompute(training_gray,None)
  mag_factor = 3
  desc_tfeat = tfeat_utils.describe_opencv(tfeat, training_gray, kp, 32,mag_factor)
  return desc_tfeat, kp

def SURF(image):
    training_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
    surf = cv2.xfeatures2d.SURF_create(800)
    keypoints, descriptor = surf.detectAndCompute(training_gray, None)
    return descriptor , keypoints

# Main def for execute the whole functions and run descriptor and matchin features and save them into text file
def Keypoint_position(database,database_name,model,model_name):
    print(f'*** Working on {database_name} Database')
    print('<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print(f'Doing with {model_name} Descriptor...')
    for filename in sorted(os.listdir(database)):  # Extract features from keypoint file
        print('Loading image: ',filename)
        img = cv2.imread(os.path.join(database,filename))
        des , kp = model(img)
        final = np.zeros((des.shape[0],128))
        points= []
        octaves= []
        angles = []
        for i in kp:
          points.append(i.pt)
          octaves.append(i.octave)
          angles.append(i.angle) 
        for i in range(len(des)): # Creat a array with 128 vectors to run in colmap(only SIFT descriptor shape acceptable in colmap)
          final[i][0] = points[i][0]
          final[i][1] = points[i][1]
          final[i][2] = octaves[i]
          final[i][3] = angles[i]
        np.savetxt(f'drive/MyDrive/3D Shabnam/{database_name}/{model_name}/Descriptors/{filename}.txt', final,fmt="%s",header=f'{final.shape[0]} {final.shape[1]}',
                   comments='') #Save the decriptors with specified numbers in a text file
        name_checker = []
        swag = []
    for filename_one in sorted(os.listdir(database)): # Doing the matcing 
      img_one = cv2.imread(os.path.join(database,filename_one))
      for filename_two in sorted(os.listdir(database)):
        if filename_one != filename_two and [filename_one,filename_two] and [filename_two,filename_one] not in name_checker:
          print(f'Matching image ---- {filename_one} with image  ---- {filename_two} ..')
          img_two = cv2.imread(os.path.join(database,filename_two))
          bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck = False)
          desc1,_ = model(img_one)
          desc2,_ = model(img_two)
          matches = bf.match(desc1, desc2)
          matches = sorted(matches, key = lambda x : x.distance)
          name_checker.append([filename_one,filename_two]) # this list skip doublicate matching in the loop
          swag.append([filename_one,filename_two])
          for k in matches:
            swag.append([k.queryIdx,k.trainIdx]) # save matching points of two image in a list and save it in txt file
          np.savetxt(f'drive/MyDrive/3D Shabnam/{database_name}/{model_name}/Matches/{filename_one},{filename_two}.txt',swag,fmt='%s')
          swag = []
    files = glob.glob(f'drive/MyDrive/3D Shabnam/{database_name}/{model_name}/Matches/*.txt') #Create a txt file with all matching
    print('Getting matches ready ...')
    print('<<<<<<<<<')
    print('>>>>>>>>>')
    with open(f'drive/MyDrive/3D Shabnam/{database_name}/{model_name}/possible_matching_{model_name}.txt', 'w') as result:
      for f in files:
        with open(f,) as infile:
          contents = infile.read()
          contents = contents + '\n'
          result.write(contents)
    print('Done!')
    print("_______________________")
    print('_______________________')
    
    # Done

In [8]:
cd /

/


In [9]:
cd content/

/content


In [ ]:
Keypoint_position(data['Database'][i],data['Database Name'][i],data['Desc'][j],data['Name'][j])

#Run Whole Program

In [ ]:
# a dict for run the program and datsa sets automaticly 
data = {'Database':[tiso,portello,fountain,castle],'Database Name':['Tiso','Portello','Fountain','Castle']
               ,'Desc': [BRIEF,ORB,TFeat,SURF], 'Name':['BRIEF','ORB','TFeat','SURF']}
for i in range(4):
  for j in range(4):
    Keypoint_position(data['Database'][i],data['Database Name'][i],data['Desc'][j],data['Name'][j])
    torch.cuda.empty_cache()

*** Working on Tiso Database
<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>
Doing with BRIEF Descriptor...
Loading image:  img00c1.jpg
Loading image:  img01c1.jpg
Loading image:  img02c1.jpg
Loading image:  img03c1.jpg
Loading image:  img04c1.jpg
Loading image:  img05c1.jpg
Loading image:  img06c1.jpg
Loading image:  img07c1.jpg
Loading image:  img08c1.jpg
Loading image:  img09c1.jpg
Loading image:  img10c2.jpg
Loading image:  img11c2.jpg
Loading image:  img12c2.jpg
Loading image:  img13c2.jpg
Loading image:  img14c2.jpg
Loading image:  img15c2.jpg
Loading image:  img16c2.jpg
Loading image:  img17c2.jpg
Loading image:  img18c2.jpg
Loading image:  img19c2.jpg
Loading image:  img20c3.jpg
Loading image:  img21c3.jpg
Loading image:  img22c3.jpg
Loading image:  img23c3.jpg
Loading image:  img24c3.jpg
Loading image:  img25c3.jpg
Loading image:  img26c3.jpg
Loading image:  img27c3.jpg
Loading image:  img28c3.jpg
Loading image:  img29c3.jpg
Matching image ---- img00c1.jpg with image  ---- img01c

RuntimeError: ignored

## Extract Matching index